In [15]:
import pandas as pd
import re
import zhon.hanzi
from collections import Counter
# import yaml as yaml

In [59]:
data = pd.read_csv("data/speech.csv")

# 使用正則表達式替換文字
data['演說'] = data['演說'].str.replace('台灣', '臺灣')

data.head(2)

,屆次,年份,黨派,總統,演說
0,16,2024,民進黨,賴清德,蕭美琴副總統、各位友邦的元首與貴賓、各國駐台使節代表、現場所有的嘉賓，電視機前、還有線上收看...
1,15,2020,民進黨,蔡英文,賴副總統、現場的各位貴賓、電視機前跟網路上的朋友、全體國人同胞，大家好。\n\n今天我站在這...


In [60]:
split_speeches = data['演說'].str.split('。\n').explode().reset_index()
data_clean = split_speeches.join(data.drop('演說', axis=1), on='index', how='left').drop('index', axis=1)[['總統', '黨派','年份','屆次', '演說']]

data_clean['演說'] = data_clean['演說'].str.replace('\n', '')

data_clean

,總統,黨派,年份,屆次,演說
0,賴清德,民進黨,2024,16,蕭美琴副總統、各位友邦的元首與貴賓、各國駐台使節代表、現場所有的嘉賓，電視機前、還有線上收看...
1,賴清德,民進黨,2024,16,我從政的時候，立志改變臺灣
2,賴清德,民進黨,2024,16,現在，站在這裡，我立志壯大國家！我以無比堅定的心情，接受人民的託付，就職中華民國第十六任總統...
3,賴清德,民進黨,2024,16,回想1949年的今天，臺灣實施戒嚴，全面進入專制的黑暗年代
4,賴清德,民進黨,2024,16,1996年的今天，臺灣第一位民選總統宣誓就職，向國際社會傳達，中華民國臺灣是一個主權獨立的國...
...,...,...,...,...,...
389,李登輝,國民黨,1996,9,去年以來，為了反對民主，中共對登輝個人發動一波又一波「欲加之罪、何患無辭」的誣衊，但是登輝忍...
390,李登輝,國民黨,1996,9,為了影響我們第一次民選總統的選情，中共進行一次又一次的軍事演習，但是我們表現了無比的自制。因...
391,李登輝,國民黨,1996,9,中華民國本來就是一個主權國家。海峽兩岸沒有民族與文化認同問題，有的只是制度與生活方式之爭。在...
392,李登輝,國民黨,1996,9,今天，登輝要鄭重呼籲：海峽兩岸，都應該正視處理結束敵對狀態這項重大問題，以便為追求國家統一的...


In [61]:
zh_pat = f"[{zhon.hanzi.characters}|{zhon.hanzi.punctuation}]"

with open("../lib/stop_word_chinese.txt", encoding="utf-8") as fin:
    stopwords = fin.read().split("\n")[0:]

stopwords[0:20]

[',',
 '\\u3000',
 '?',
 '、',
 '。',
 '“',
 '”',
 '，',
 '。',
 '！',
 '？',
 '；',
 '：',
 '“',
 '”',
 '「',
 '」',
 '■',
 '▲',
 ' ▲']

In [57]:
# import unicodedata
# def remove_punc_by_unicode(words):
#     out = []
#     for word in words:
#         if word == " " or  unicodedata.category(word[0]).startswith('P'):
#             continue
#         out.append(word)
#     return out

import unicodedata
def remove_punc_by_unicode(article):
    out = ""
    for word in article:
        if word != " " and not unicodedata.category(word[0]).startswith('P'):
            out += word
    return out

In [62]:
data_clean['演說_removePunc'] = data_clean['演說'].apply(remove_punc_by_unicode)
data_clean

,總統,黨派,年份,屆次,演說,演說_removePunc
0,賴清德,民進黨,2024,16,蕭美琴副總統、各位友邦的元首與貴賓、各國駐台使節代表、現場所有的嘉賓，電視機前、還有線上收看...,蕭美琴副總統各位友邦的元首與貴賓各國駐台使節代表現場所有的嘉賓電視機前還有線上收看直播的好朋...
1,賴清德,民進黨,2024,16,我從政的時候，立志改變臺灣,我從政的時候立志改變臺灣
2,賴清德,民進黨,2024,16,現在，站在這裡，我立志壯大國家！我以無比堅定的心情，接受人民的託付，就職中華民國第十六任總統...,現在站在這裡我立志壯大國家我以無比堅定的心情接受人民的託付就職中華民國第十六任總統我將依據中...
3,賴清德,民進黨,2024,16,回想1949年的今天，臺灣實施戒嚴，全面進入專制的黑暗年代,回想1949年的今天臺灣實施戒嚴全面進入專制的黑暗年代
4,賴清德,民進黨,2024,16,1996年的今天，臺灣第一位民選總統宣誓就職，向國際社會傳達，中華民國臺灣是一個主權獨立的國...,1996年的今天臺灣第一位民選總統宣誓就職向國際社會傳達中華民國臺灣是一個主權獨立的國家主權在民
...,...,...,...,...,...,...
389,李登輝,國民黨,1996,9,去年以來，為了反對民主，中共對登輝個人發動一波又一波「欲加之罪、何患無辭」的誣衊，但是登輝忍...,去年以來為了反對民主中共對登輝個人發動一波又一波欲加之罪何患無辭的誣衊但是登輝忍辱負重不予理...
390,李登輝,國民黨,1996,9,為了影響我們第一次民選總統的選情，中共進行一次又一次的軍事演習，但是我們表現了無比的自制。因...,為了影響我們第一次民選總統的選情中共進行一次又一次的軍事演習但是我們表現了無比的自制因為我們...
391,李登輝,國民黨,1996,9,中華民國本來就是一個主權國家。海峽兩岸沒有民族與文化認同問題，有的只是制度與生活方式之爭。在...,中華民國本來就是一個主權國家海峽兩岸沒有民族與文化認同問題有的只是制度與生活方式之爭在這裡我...
392,李登輝,國民黨,1996,9,今天，登輝要鄭重呼籲：海峽兩岸，都應該正視處理結束敵對狀態這項重大問題，以便為追求國家統一的...,今天登輝要鄭重呼籲海峽兩岸都應該正視處理結束敵對狀態這項重大問題以便為追求國家統一的歷史大業...


In [63]:
# ws_driver  = CkipWordSegmenter(model="bert-base", device=torch.device('mps')) # 用GPU會出下面那句error
ws_driver  = CkipWordSegmenter(model="bert-base", device=-1)
# pos_driver = CkipPosTagger(model="bert-base", device=-1)



In [65]:
data_clean['tokens']  = ws_driver(data_clean['演說'])

Inference: 100%|█████████████████████████████████| 2/2 [03:34<00:00, 107.09s/it]


In [67]:
print(data_clean['tokens'].iloc[8])

['一', '、', '行政', '立法', '協調', '合作', '，', '共同', '推動', '國政', '今年', '二月', '上任', '的', '新', '國會', '，', '是', '臺灣', '時隔', '十六', '年', '後', '，', '再次', '出現', '「', '三', '黨', '不', '過半', '」', '的', '立法院', '。', '面對', '這', '個', '政治', '新局', '，', '有些', '人', '抱持', '期待', '，', '也', '有些', '人', '感到', '憂心']


In [75]:
data_clean

,總統,黨派,年份,屆次,演說,演說_removePunc,tokens
0,賴清德,民進黨,2024,16,蕭美琴副總統、各位友邦的元首與貴賓、各國駐台使節代表、現場所有的嘉賓，電視機前、還有線上收看...,蕭美琴副總統各位友邦的元首與貴賓各國駐台使節代表現場所有的嘉賓電視機前還有線上收看直播的好朋...,"[蕭美琴, 副總統, 、, 各位, 友邦, 的, 元首, 與, 貴賓, 、, 各, 國, 駐..."
1,賴清德,民進黨,2024,16,我從政的時候，立志改變臺灣,我從政的時候立志改變臺灣,"[我, 從政, 的, 時候, ，, 立志, 改變, 臺灣]"
2,賴清德,民進黨,2024,16,現在，站在這裡，我立志壯大國家！我以無比堅定的心情，接受人民的託付，就職中華民國第十六任總統...,現在站在這裡我立志壯大國家我以無比堅定的心情接受人民的託付就職中華民國第十六任總統我將依據中...,"[現在, ，, 站, 在, 這裡, ，, 我, 立志, 壯大, 國家, ！, 我, 以, 無..."
3,賴清德,民進黨,2024,16,回想1949年的今天，臺灣實施戒嚴，全面進入專制的黑暗年代,回想1949年的今天臺灣實施戒嚴全面進入專制的黑暗年代,"[回想, 1949年, 的, 今天, ，, 臺灣, 實施, 戒嚴, ，, 全面, 進入, 專..."
4,賴清德,民進黨,2024,16,1996年的今天，臺灣第一位民選總統宣誓就職，向國際社會傳達，中華民國臺灣是一個主權獨立的國...,1996年的今天臺灣第一位民選總統宣誓就職向國際社會傳達中華民國臺灣是一個主權獨立的國家主權在民,"[1996年, 的, 今天, ，, 臺灣, 第一, 位, 民選, 總統, 宣誓, 就職, ，..."
...,...,...,...,...,...,...,...
389,李登輝,國民黨,1996,9,去年以來，為了反對民主，中共對登輝個人發動一波又一波「欲加之罪、何患無辭」的誣衊，但是登輝忍...,去年以來為了反對民主中共對登輝個人發動一波又一波欲加之罪何患無辭的誣衊但是登輝忍辱負重不予理...,"[去年, 以來, ，, 為了, 反對, 民主, ，, 中共, 對, 登輝, 個人, 發動, ..."
390,李登輝,國民黨,1996,9,為了影響我們第一次民選總統的選情，中共進行一次又一次的軍事演習，但是我們表現了無比的自制。因...,為了影響我們第一次民選總統的選情中共進行一次又一次的軍事演習但是我們表現了無比的自制因為我們...,"[為, 了, 影響, 我們, 第一, 次, 民選, 總統, 的, 選情, ，, 中共, 進行..."
391,李登輝,國民黨,1996,9,中華民國本來就是一個主權國家。海峽兩岸沒有民族與文化認同問題，有的只是制度與生活方式之爭。在...,中華民國本來就是一個主權國家海峽兩岸沒有民族與文化認同問題有的只是制度與生活方式之爭在這裡我...,"[中華民國, 本來, 就, 是, 一, 個, 主權, 國家, 。, 海峽, 兩岸, 沒有, ..."
392,李登輝,國民黨,1996,9,今天，登輝要鄭重呼籲：海峽兩岸，都應該正視處理結束敵對狀態這項重大問題，以便為追求國家統一的...,今天登輝要鄭重呼籲海峽兩岸都應該正視處理結束敵對狀態這項重大問題以便為追求國家統一的歷史大業...,"[今天, ，, 登輝, 要, 鄭重, 呼籲, ：, 海峽, 兩岸, ，, 都, 應該, 正視..."


In [71]:
data_clean.to_csv("data/speech_token_ckipraw.csv")
data_clean.to_pickle("data/speech_token_ckipraw.pkl")


In [123]:
tokens = pd.DataFrame(data_clean['tokens'].explode()).reset_index().rename(columns={'tokens': 'words'})
# data_clean = data_clean.reset_index()

data_tokenized = pd.merge(tokens, data_clean.reset_index(), on='index', how='left')[['index', '總統', '黨派', '年份', '屆次', '演說', '演說_removePunc', 'tokens', 'words']]


In [124]:
data_tokenized.to_csv("data/speech_tokenized.csv")
data_tokenized.to_pickle("data/speech_tokenized.pkl")
